## Example SpikeForest import

This notebook will import a few SpikeForest recordings into the labbox_ephys database

In [ ]:
# Define some functions

import lbdb
import kachery as ka
from labbox_ephys import AutoRecordingExtractor, AutoSortingExtractor
import hither
import numpy as np

def geom_from_recording(recording):
    channel_ids = recording.get_channel_ids()
    location0 = recording.get_channel_property(channel_ids[0], 'location')
    nd = len(location0)
    M = len(channel_ids)
    geom = np.zeros((M, nd))
    for ii in range(len(channel_ids)):
        location_ii = recording.get_channel_property(channel_ids[ii], 'location')
        geom[ii, :] = list(location_ii)
    return geom

def import_recording(*, recording_id, recording_path):
    print(f'Importing {recording_id}')
    recording = AutoRecordingExtractor(recording_path, download=True)
    doc = dict(
        recording_id=recording_id,
        recording_path=recording_path,
        sampling_frequency=recording.get_sampling_frequency(),
        channel_ids=recording.get_channel_ids(),
        geom=geom_from_recording(recording).tolist(),
        num_frames=recording.get_num_frames()
    )
    lbdb.update(collection='recordings', query=dict(recording_id=recording_id), update=doc, upsert=True)

def import_sorting(*, recording_id, recording_path, sorting_name, sorting_path):
    print(f'Importing {recording_id}::{sorting_name}')
    sorting = AutoSortingExtractor(sorting_path)
    doc = dict(
        recording_id=recording_id,
        recording_path=recording_path,
        sorting_path=sorting_path,
        sorting_name=sorting_name,
        sorting_id=recording_id + '::' + sorting_name,
        unit_ids=sorting.get_unit_ids().tolist()
    )
    lbdb.update(collection='sortings', query=dict(recording_id=recording_id, sorting_name=sorting_name), update=doc, upsert=True)

def import_recording_with_truth(recording_id, recording_path, sorting_true_path):
    import_recording(recording_id=recording_id, recording_path=recording_path)
    import_sorting(recording_id=recording_id, recording_path=recording_path, sorting_name='truth', sorting_path=sorting_true_path)

In [ ]:
# Import some example SpikeForest recordings

with ka.config(fr='default_readonly'):
    # SYNTH_MAGLAND example
    import_recording_with_truth(
        recording_id='sf:synth_magland_noise10_K10_C4/001_synth',
        recording_path='sha1://ee5214337b2e01910a92c3613a4b8ad4be4dc476/SYNTH_MAGLAND/synth_magland_noise10_K10_C4/001_synth.json',
        sorting_true_path='sha1://ff64f1713227c017052bf21b41ddf764320aa606/SYNTH_MAGLAND/synth_magland_noise10_K10_C4/001_synth.firings_true.json'
    )
    
    # PAIRED_KAMPFF example
    import_recording_with_truth(
        recording_id='sf:paired_kampff/2014_11_25_Pair_3_0',
        recording_path='sha1://b96e34b72c258be205e1bf2a7aaaad72dfd8e351/PAIRED_KAMPFF/paired_kampff/2014_11_25_Pair_3_0.json',
        sorting_true_path='sha1://86f472cba745921dabb9b4e745f2e3547f9fb3e9/PAIRED_KAMPFF/paired_kampff/2014_11_25_Pair_3_0.firings_true.json'
    )
    
    # PAIRED_BOYDEN example
    import_recording_with_truth(
        recording_id='sf:paired_boyden32c/1103_1_1',
        recording_path='sha1://0c89b66696ff92e529d97a31fe41af7949111a9a/PAIRED_BOYDEN/paired_boyden32c/1103_1_1.json',
        sorting_true_path='sha1://fe3f7c918200ffe44db7759911e80978540d5201/PAIRED_BOYDEN/paired_boyden32c/1103_1_1.firings_true.json'
    )

In [ ]:
# List all of the recordings in the database

recordings = lbdb.find(collection='recordings', query=dict())
for rec in recordings:
    print(rec['recording_id'])

In [ ]:
# List all of the sortings in the database

sortings = lbdb.find(collection='sortings', query=dict())
for rec in sortings:
    print(rec['sorting_id'])